<a href="https://colab.research.google.com/github/NatLibFi/Annif/blob/kvp2019-workshop/arviointipohjien_luonti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notebook for creating templates for Kirjastoverkkopäivät 2019

- Fetches subjects for documents from a folder in Google Drive (where each spreadsheet file contains subjects multiple documents)
- Creates templates for reviewers in another folder in Google Drive (one spreadsheet file for each reviewer, where one sheet/tab corresponds to a document, and description)


In [0]:
! pip install gspread-pandas gspread-formatting

In [0]:
! mkdir -p  /root/.config/gspread_pandas/
! echo '{"installed":{"client_id":"257572009167-uai362klp88kr3hluabtoujf2hnd7fr6.apps.googleusercontent.com","project_id":"formal-archive-254810","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"x554t-fVXmFDrENKrXu8GuKy","redirect_uris":["urn:ietf:wg:oauth:2.0:oob","http://localhost"]}}' > /root/.config/gspread_pandas/google_secret.json

In [0]:
import pandas as pd
import random
from gspread_pandas import Spread, Client
from gspread_formatting import cellFormat, format_cell_range, set_frozen, color, textFormat

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 100) 

In [29]:
%%time
# Fetch kuvailut
folder = 'Dokumenttien kuvailut'

client = Client()
spreads = client.find_spreadsheet_files_in_folders(folder)

df_kuvailut1 = pd.DataFrame()
df_kuvailut2 = pd.DataFrame()
df_kuvailut3 = pd.DataFrame()

for kuvailu_id, spread in enumerate(spreads[folder]):
  print(spread['name'])
  # Tietokirjat:
  if spread['name'].startswith("Kuvailut Annif"):
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.columns = df_sheet.columns.str.lower()
      df_kuvailut1 = df_kuvailut1.append(df_sheet)
  elif spread['name'].startswith("kuvailut_tieto"):
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.rename(columns={'isbn': 'url kokotekstiin'}, inplace=True)
      df_sheet.columns = df_sheet.columns.str.lower()
      df_kuvailut2 = df_kuvailut2.append(df_sheet)
  else:
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.columns = df_sheet.columns.str.lower()
      df_kuvailut3 = df_kuvailut3.append(df_sheet)    
  

SKT3 JM Copy of Kuvailut
SKT1 KL Copy of Kuvailut
Copy of Kuvailut Fennica


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jyx_kuvailut Osma
SKT2 PM Copy of Kuvailut 
SKT4 VL Copy of Kuvailut 
Copy of Kuvailut K27
Kuvailut Annif-FastText
Kuvailut Annif-PAV
Kuvailut Annif-TFIDF
Kuvailut Annif-Maui
jyx_kuvailut Juho
jyx_kuvailut Mona
jyx_kuvailut_orig
kuvailut_tieto Osma
Copy of Kuvailut K07
Copy of Kuvailut K16
Copy of Kuvailut K31
kuvailut_tieto Mona
kuvailut_tieto Fennica
kuvailut_tieto Juho
Copy of Kuvailut K02
Copy of Kuvailut K17
Copy of Kuvailut K05
Copy of Kuvailut K14
Copy of Kuvailut K11
Copy of Kuvailut K08
Copy of Kuvailut K32
Copy of Kuvailut K23
Copy of Kuvailut K29
Copy of Kuvailut K20
Copy of Kuvailut K26
Copy of Kuvailut K19
Copy of Kuvailut K10
Copy of Kuvailut K38
Copy of Kuvailut K13
Copy of Kuvailut K35
Copy of Kuvailut K40
Copy of Kuvailut K28
Copy of Kuvailut K25
Copy of Kuvailut K37
Copy of Kuvailut K22
Copy of Kuvailut K18
Copy of Kuvailut K06
Copy of Kuvailut K12
Copy of Kuvailut K03
Copy of Kuvailut K09
Copy of Kuvailut K33
Copy of Kuvailut K39
Copy of Kuvailut K21
Copy of Kuvailut

In [58]:
### Preprocess

df_kuvailut = pd.concat([df_kuvailut2, df_kuvailut3])
cols_to_keep = ['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi']
df_kuvailut = df_kuvailut[cols_to_keep]
df_kuvailut.rename(columns={'id': 'did'}, inplace=True)
df_kuvailut[['did', 'nimeke', 'url kokotekstiin']] = df_kuvailut[['did', 'nimeke', 'url kokotekstiin']].mask(df_kuvailut[['did', 'nimeke', 'url kokotekstiin']]=='', None).ffill()

# Append Annif indexed, and merge nimeke and url to them 
df_kuvailut1.rename(columns={'id': 'did'}, inplace=True)
df_kuvailut1['did'] = df_kuvailut1['did'].mask(df_kuvailut1['did']=='', None).ffill()
df_kuvailut = df_kuvailut.append(df_kuvailut1)

df_kuvailut = df_kuvailut[df_kuvailut.aiheet != '']

# Restrict documents to those in the list file:
df_documents = Spread('Arvioitavat dokumentit', sheet=0).sheet_to_df(index=None)
print('Document IDs that are removed:')
display(df_kuvailut[~df_kuvailut.did.isin(df_documents.did)].did.unique())
df_kuvailut = df_kuvailut[df_kuvailut.did.isin(df_documents.did)]

# Merge nimeke and url from the list file (to avoid duplicated urls (and correct url for D20)):
df_kuvailut = df_kuvailut.drop(columns=['nimeke', 'url kokotekstiin']).merge(df_documents[['did', 'nimeke', 'url kokotekstiin']], on='did')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Document IDs that are removed:


array(['SKT14', 'SKT23', 'SKT03', 'SKT44', 'SKT50', 'SKT17', 'SKT43',
       'SKT29', 'SKT39', 'SKT31', 'SKT47', 'SKT16', 'SKT07', 'SKT22',
       'SKT21', 'SKT28', 'SKT35', 'SKT48', 'SKT45', 'SKT42', 'SKT12',
       'SKT20', 'SKT06', 'SKT09', 'SKT32', 'SKT10', 'SKT49', 'SKT36',
       'SKT25', 'SKT38', 'SKT30', 'SKT26', 'SKT15', 'SKT34', 'SKT05',
       'SKT33', 'SKT01', 'SKT04', 'SKT08', 'SKT11', 'D00'], dtype=object)

In [0]:
#df = df_kuvailut.drop(columns=['url kokotekstiin']).merge(df_documents[['did', 'url kokotekstiin']], on='did')
#df
#df_kuvailut


In [59]:
# Set types of index source method

def insert_type_column(df, column_name, source_type, type_file_names):
    rows = df.tiedostonimi.isin(type_file_names)
    df.loc[rows, column_name] = source_type
    return df   


files_annif_assisted = ['kuvailut_tieto Osma', 'kuvailut_tieto Juho', 'jyx_kuvailut_orig', 'jyx_kuvailut Osma', ]
files_annif = ['Kuvailut Annif-FastText', 'Kuvailut Annif-Maui', 'Kuvailut Annif-PAV', 'Kuvailut Annif-TFIDF', ]
files_human = ['kuvailut_tieto Mona', 'kuvailut_tieto Fennica',
               'jyx_kuvailut Mona', 'jyx_kuvailut Juho', 
                'SKT1 KL Copy of Kuvailut',  'SKT2 PM Copy of Kuvailut',  'SKT3 JM Copy of Kuvailut',  'SKT4 VL Copy of Kuvailut', 
               'Copy of Kuvailut Fennica', ] + ['Copy of Kuvailut K{:02d}'.format(x) for x in range(1, 41)]

df_kuvailut = insert_type_column(df_kuvailut, 'tyyppi', 'annif_avusteinen', files_annif_assisted)
df_kuvailut = insert_type_column(df_kuvailut, 'tyyppi', 'ihminen', files_human)
df_kuvailut = insert_type_column(df_kuvailut, 'tyyppi', 'annif', files_annif)


# Subtypes:
files_fennica = ['kuvailut_tieto Fennica', 'Copy of Kuvailut Fennica']

df_kuvailut = insert_type_column(df_kuvailut, 'alatyyppi', 'ihminen', files_human)
df_kuvailut = insert_type_column(df_kuvailut, 'alatyyppi', 'annif_avusteinen', files_annif_assisted)

df_kuvailut = insert_type_column(df_kuvailut, 'alatyyppi', 'FastText', ['Kuvailut Annif-FastText'])
df_kuvailut = insert_type_column(df_kuvailut, 'alatyyppi', 'Maui', ['Kuvailut Annif-Maui'])
df_kuvailut = insert_type_column(df_kuvailut, 'alatyyppi', 'PAV', ['Kuvailut Annif-PAV'])
df_kuvailut = insert_type_column(df_kuvailut, 'alatyyppi', 'TFIDF', ['Kuvailut Annif-TFIDF'])
df_kuvailut = insert_type_column(df_kuvailut, 'alatyyppi', 'fennica', files_fennica)

df_kuvailut

,aiheet,did,kid,tiedostonimi,nimeke,url kokotekstiin,tyyppi,alatyyppi
0,virheet,TI01,14,kuvailut_tieto Osma,"Mahtava moka : uskalla, opi ja menesty",9789521433085,annif_avusteinen,annif_avusteinen
1,epäonnistuminen,TI01,14,kuvailut_tieto Osma,"Mahtava moka : uskalla, opi ja menesty",9789521433085,annif_avusteinen,annif_avusteinen
2,johtaminen,TI01,14,kuvailut_tieto Osma,"Mahtava moka : uskalla, opi ja menesty",9789521433085,annif_avusteinen,annif_avusteinen
3,pelko,TI01,14,kuvailut_tieto Osma,"Mahtava moka : uskalla, opi ja menesty",9789521433085,annif_avusteinen,annif_avusteinen
4,esimiehet,TI01,14,kuvailut_tieto Osma,"Mahtava moka : uskalla, opi ja menesty",9789521433085,annif_avusteinen,annif_avusteinen
5,organisaatiokulttuuri,TI01,14,kuvailut_tieto Osma,"Mahtava moka : uskalla, opi ja menesty",9789521433085,annif_avusteinen,annif_avusteinen
6,rohkeus,TI01,14,kuvailut_tieto Osma,"Mahtava moka : uskalla, opi ja menesty",9789521433085,annif_avusteinen,annif_avusteinen
7,erehdys,TI01,18,kuvailut_tieto Mona,"Mahtava moka : uskalla, opi ja menesty",9789521433085,ihminen,ihminen
8,virheet,TI01,18,kuvailut_tieto Mona,"Mahtava moka : uskalla, opi ja menesty",9789521433085,ihminen,ihminen
9,työelämätaidot,TI01,18,kuvailut_tieto Mona,"Mahtava moka : uskalla, opi ja menesty",9789521433085,ihminen,ihminen


In [60]:
# Create header and footer to insert around kuvailu 

df_ylaosa = pd.DataFrame({'did': [''], 'kid': [''], 'aiheet': ['aiheet'], 'poistettavat': ['poista = x'], 'empty1': [''], 'empty2': ['']})

df_alaosa0 = pd.DataFrame({'did': [''], 'aiheet': [''], 'kid': ['']})
df_alaosa1 = pd.DataFrame({'did': [''], 'empty2': ['arvosana (1-5)'], 'kid': ['']})
df_alaosa2 = pd.DataFrame({'did': ['']*4, 'empty1': ['Kuvailu on kattava', 'Käytetyt asiasanat ovat sisällön kannalta merkityksellisiä', 'Kuvailu on laadukas', 'Vapaa kenttä'], 'kid': ['']*4})
df_alaosa = pd.concat([df_alaosa0, df_alaosa1, df_alaosa2, df_alaosa0, df_alaosa0])


display(df_ylaosa)
display(df_alaosa)

# Formatting
fmt = cellFormat(
    horizontalAlignment='RIGHT'
    )
fmt_header = cellFormat(
    backgroundColor=color(144/256., 238/256., 144/256.),
    textFormat=textFormat(bold=True) #, foregroundColor=color(1, 0, 1)),
    )


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,did,kid,aiheet,poistettavat,empty1,empty2
0,,,aiheet,poista = x,,


,aiheet,did,empty1,empty2,kid
0,,,NaN,NaN,
0,NaN,,NaN,arvosana (1-5),
0,NaN,,Kuvailu on kattava,NaN,
1,NaN,,Käytetyt asiasanat ovat sisällön kannalta merkityksellisiä,NaN,
2,NaN,,Kuvailu on laadukas,NaN,
3,NaN,,Vapaa kenttä,NaN,
0,,,NaN,NaN,
0,,,NaN,NaN,


In [61]:
# Save kuvailut data for use in analysis

from google.colab import drive
drive.mount('drive')

df_kuvailut.to_csv('kuvailut.csv')
!cp kuvailut.csv drive/My\ Drive/Kivepa2019\ Aaveita\ koneessa\ \(kivepa-aaveita\)/


Mounted at drive


In [0]:
%%time
# Create files and sheets
folder_to_save = 'Kivepa2019 Aaveita koneessa (kivepa-aaveita)/Arvioijakohtaiset taulukot'
min_num_aiheet = 1
max_num_ihmiskuvailut = 3
num_arvioija =2  # One file for one reviewer
num_sheets = 50  # One sheet for one document

include_types = ['annif', 'annif_avusteinen']
columns = ['did', 'kid', 'aiheet'] #, 'tiedostonimi']

dokumentit = [(did, df_dokumentti) for did, df_dokumentti in df_kuvailut.groupby('did')]

for arvioija_ind in range(num_arvioija):
    print('arvioija_ind: ', arvioija_ind)
    spread = Spread('arvioija_' + str(arvioija_ind) , create_spread=True)
    spread.move(folder_to_save)
    #random.shuffle(dokumentit)

    cnt_sheet = 0
    for did, df_dokumentti in dokumentit:
        spread.open_sheet(did, create=True)
        df_pohja = pd.DataFrame(columns=columns)
        print('document_id: ', did)
        # First get kuvailut that are allways included:
        kuvailut = [df_dokumentti_kuvailu 
                    for _, df_dokumentti_kuvailu 
                    in df_dokumentti.groupby('kid')
                    if (df_dokumentti_kuvailu.tyyppi.iloc[0] in include_types
                    or df_dokumentti_kuvailu.alatyyppi.iloc[0] == 'fennica')
                    and len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) >= min_num_aiheet
                    ]
        # Then get at most max_num_ihmiskuvailut normal human-made kuvailua (not fennica):
        ihmiskuvailut = [df_dokumentti_kuvailu 
                    for _, df_dokumentti_kuvailu 
                    in df_dokumentti.groupby('kid')
                    if df_dokumentti_kuvailu.alatyyppi.iloc[0] == 'ihminen'
                    and len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) >= min_num_aiheet
                    ]
        random.shuffle(ihmiskuvailut)
        ihmiskuvailut = ihmiskuvailut[:max_num_ihmiskuvailut]
        kuvailut.extend(ihmiskuvailut)
        random.shuffle(kuvailut)
        
        df_header = df_dokumentti.iloc[0][['did', 'nimeke', 'url kokotekstiin']].to_frame()
        spread.df_to_sheet(df_header, headers=False, index=False)

        for df_dokumentti_kuvailu in kuvailut:
            df_pohja = df_pohja.append(df_ylaosa.append(df_dokumentti_kuvailu[columns], sort=False).append(df_alaosa, sort=False).fillna(''), sort=False)

        spread.df_to_sheet(df_pohja, index=False, headers=False, start=(4, 1))
        format_cell_range(spread.sheet, 'D1:E' + str(len(df_pohja)), fmt)
        set_frozen(spread.sheet, rows=3)
        format_cell_range(spread.sheet, 'A1:F3', fmt_header)
        
        cnt_sheet += 1
        if cnt_sheet >= num_sheets:
            break
    spread.delete_sheet('Sheet1')
    dokumentit = dokumentit[1:] + [dokumentit[0]]  # Rotate order from front to back


arvioija_ind:  0
document_id:  D01
document_id:  D02
document_id:  D03
document_id:  D04
document_id:  D05
document_id:  D06
document_id:  D07
document_id:  D08
document_id:  D09
document_id:  D10
document_id:  D11
document_id:  D12
document_id:  D13
document_id:  D14
document_id:  D15
document_id:  D16
document_id:  D17
document_id:  D18
document_id:  D19
document_id:  D20
document_id:  JYX01
document_id:  JYX02
document_id:  JYX03
document_id:  JYX04
document_id:  JYX05
document_id:  JYX06
document_id:  JYX07
document_id:  JYX08
document_id:  JYX09
document_id:  JYX10
document_id:  SKT02
document_id:  SKT13
document_id:  SKT18
document_id:  SKT19
document_id:  SKT24
document_id:  SKT27
document_id:  SKT37
document_id:  SKT40
document_id:  SKT41
document_id:  SKT46
document_id:  TI01
document_id:  TI02
document_id:  TI03
document_id:  TI04
document_id:  TI05
document_id:  TI06
document_id:  TI07
document_id:  TI08
document_id:  TI09
document_id:  TI10
arvioija_ind:  1
document_id:  D0